In [ ]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from skimage.draw import polygon
from PIL import Image, ImageDraw
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
import time

import sklearn.metrics as metrics
from sklearn.metrics import roc_auc_score
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

from DSCLChecker.dscl import *
from DSCLChecker.monitor import *

In [3]:
def visualize_regions(image, region_info):
    H, W = image.shape[:2]
    label_map = np.zeros((H, W), dtype=int)
    for region_id, info in region_info.items():
        print(f"Region ID: {region_id}, Area: {len(info['pixels'])}")
        if len(info['pixels']) == 1:
            print(info['pixels'])
        for (r, c) in info['pixels']:
            label_map[r, c] = region_id

    plt.figure(figsize=(6, 6))
    plt.imshow(image, cmap='gray', interpolation='nearest')
    plt.imshow(label_map, cmap='tab10', alpha=0.5, interpolation='nearest')
    plt.axis('off')
    plt.title("Region Overlay")
    plt.show()

In [ ]:
mask_path = "OCTA-Dataset/2D Baselines/logs/test_results"
prob_path = "OCTA-Dataset/2D Baselines/logs/test_visuals"
gt_path = "OCTA-Dataset/Label/GT_LargeVesse_test"

def load_images_as_np(folder):
    all_images = {}
    for fname in os.listdir(folder):
        fpath = os.path.join(folder, fname)
        if fname.endswith('.npy'):
            arr = np.load(fpath)
        elif fname.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif')):
            arr = np.array(Image.open(fpath).convert("L"))
        else:
            continue
        all_images[fname] = arr
    return all_images

mask_images = load_images_as_np(mask_path)
prob_images = load_images_as_np(prob_path)
gt_images = load_images_as_np(gt_path)

print(f"Loaded {len(mask_images)} mask images")
print(f"Loaded {len(prob_images)} probability images")
print(f"Loaded {len(gt_images)} ground truth images")

img_path = "OCTA-Dataset/Projection Maps/OCTA(ILM_OPL)"

original_images = load_images_as_np(img_path)
print(f"Loaded {len(original_images)} mask images")


Loaded 20 mask images
Loaded 20 probability images
Loaded 20 ground truth images
Loaded 200 mask images


In [6]:
processed_images = [] 

for fname, seg_mask in mask_images.items():

    scaled_prob_image = prob_images[fname]/255.0
    scaled_original_image = original_images[fname]/255.0
    scaled_mask_image = seg_mask/255.0
    
    region_info = extract_component_bounds(seg_mask)
    region_map = np.zeros((304, 304), dtype=np.uint8)
    for region in region_info.values():
        for (r, c) in region['pixels']:
            region_map[r, c] = region['region_id']
    
    combined = np.zeros((304, 304, 4), dtype=np.float32)
    combined[:, :, 0] = region_map.astype(np.uint8) 
    combined[:, :, 1] = scaled_mask_image.astype(np.uint8)
    combined[:, :, 2] = scaled_prob_image.astype(np.float32)
    combined[:, :, 3] = scaled_original_image.astype(np.float32)

    processed_images.append(combined)

In [ ]:
resized_images = []

for img in processed_images:
    resized_img = np.zeros((64, 64, 4), dtype=img.dtype)
    for ch in range(4):
        resized_img[:, :, ch] = cv2.resize(
            img[:, :, ch],
            (64, 64),
            interpolation=cv2.INTER_NEAREST
        )
    resized_images.append(resized_img)

In [ ]:
total_regions = []
violated_regions = []
violated_region_id = []

monitor_times_per_spec = []
total_regions = []

for combined in processed_images: 

    region_info = extract_component_bounds(combined[:,:,1])
    total_regions.append(len(region_info))
    spec_times = []
    
    total_regions.append(len(region_info))
    violated_region_count = 0
    violated_region_id_per_image = []
    
    for region_id, info in region_info.items():
        r_tl = region_info[region_id]['top_left']
        r_br = region_info[region_id]['bottom_right']
        r_tr = region_info[region_id]['top_right']
        r_bl = region_info[region_id]['bottom_left']

        logic_border_r0 = "exists ({}, {}, {}, {}) p.row <= 0".format(r_tl, r_tr, r_bl, r_br)
        logic_border_c0 = "exists ({}, {}, {}, {}) p.col <= 0".format(r_tl, r_tr, r_bl, r_br)
        logic_border_rl = "exists ({}, {}, {}, {}) p.row >= 303".format(r_tl, r_tr, r_bl, r_br)
        logic_border_cl = "exists ({}, {}, {}, {}) p.col >= 303".format(r_tl, r_tr, r_bl, r_br)
        border_checking = "((({} | {}) | {}) | {} )".format(logic_border_r0, logic_border_c0, logic_border_rl, logic_border_cl)
        
        parsed = parse(border_checking) # change here for different logic
        start_time = time.time()
        parsed_result_bool = quantitativescore(parsed, combined)

        elapsed = time.time() - start_time
        spec_times.append(elapsed)
        violated_region_count += 1-int(parsed_result_bool)
        if not parsed_result_bool:
            violated_region_id_per_image.append(region_id)
    
    violated_regions.append(violated_region_count)
    violated_region_id.append(violated_region_id_per_image)
    
    if spec_times:
        avg_time_image = np.mean(spec_times)
        monitor_times_per_spec.append(avg_time_image)
    
print(f"Total Regions: {sum(total_regions)}")
print(f"Violated Regions: {sum(violated_regions)}")
print(sum(violated_regions)/sum(total_regions))

avg_time = np.mean(monitor_times_per_spec)
std_time = np.std(monitor_times_per_spec)
avg_regions = np.mean(total_regions)
std_regions = np.std(total_regions)

print(f"Average monitoring time per spec per image: {avg_time:.6f} seconds")
print(f"Standard deviation: {std_time:.6f} seconds")
print(f"Avg number of regions per image: {avg_regions:.2f}")
print(f"Std of regions per image: {std_regions:.2f}")

Total Regions: 1216
Violated Regions: 7858
6.462171052631579
Average monitoring time per spec per image: 0.220863 seconds
Standard deviation: 0.011265 seconds
Avg number of regions per image: 30.40
Std of regions per image: 5.03


In [37]:
def compute_dice(pred, gt):
    intersection = np.sum(pred * gt)
    return (2.0 * intersection) / (np.sum(pred) + np.sum(gt) + 1e-8)

def compute_iou(pred, gt):
    intersection = np.sum(pred * gt)
    union = np.sum((pred + gt) > 0)
    return intersection / (union + 1e-8)

dice_scores = []
iou_scores = []

for fname in gt_images:
    pred_mask = mask_images[fname]
    gt_mask = gt_images[fname]
    
    pred_binary = (pred_mask > 127).astype(np.uint8)
    gt_binary = (gt_mask > 127).astype(np.uint8)

    dice = compute_dice(pred_binary, gt_binary)
    iou = compute_iou(pred_binary, gt_binary)

    dice_scores.append(dice)
    iou_scores.append(iou)

print(f"Mean Dice: {np.mean(dice_scores):.4f}")
print(f"Mean IoU: {np.mean(iou_scores):.4f}")
print(f"Median Dice: {np.median(dice_scores):.4f}")
print(f"Median IoU: {np.median(iou_scores):.4f}")

Mean Dice: 0.9099
Mean IoU: 0.8353
Median Dice: 0.9165
Median IoU: 0.8459
